In [1]:
from options.options import Options
import os
import torch
from build_dataset_model import build_loaders, build_model
from utils import get_model_attr, calculate_model_losses, tensor_aug
from collections import defaultdict
import math

from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import numpy as np

from new.CustomVAE import *
from utils import calculate_model_losses

In [2]:
args = Options().parse()
if (args.output_dir is not None) and (not os.path.isdir(args.output_dir)):
    os.mkdir(args.output_dir)
if (args.test_dir is not None) and (not os.path.isdir(args.test_dir)):
    os.mkdir(args.test_dir)

# no KL divergence loss
args.use_AE = True

# tensorboard
# writer = SummaryWriter()

# load data
vocab, train_loader, val_loader = build_loaders(args)

dt = train_loader.dataset

| options
dataset: suncg
suncg_train_dir: metadata/data_rot_train.json
suncg_val_dir: metadata/data_rot_val.json
suncg_data_dir: /home/yizhou/Research/SUNCG/suncg_data
loader_num_workers: 1
embedding_dim: 64
gconv_mode: feedforward
gconv_dim: 128
gconv_hidden_dim: 512
gconv_num_layers: 5
mlp_normalization: batch
vec_noise_dim: 0
layout_noise_dim: 32
batch_size: 16
num_iterations: 20000
eval_mode_after: -1
learning_rate: 0.0001
print_every: 100
checkpoint_every: 1000
snapshot_every: 10000
output_dir: ./checkpoints
checkpoint_name: latest_checkpoint
timing: False
multigpu: False
restore_from_checkpoint: False
checkpoint_start_from: None
test_dir: ./layouts_out
gpu_id: 0
KL_loss_weight: 0.1
use_AE: False
decoder_cat: True
train_3d: True
KL_linear_decay: False
use_attr_30: True
manual_seed: 42
batch_gen: False
measure_acc_l1_std: False
heat_map: False
draw_2d: False
draw_3d: False
fine_tune: False
gan_shade: False
blender_path: /home/yizhou/blender-2.92.0-linux64/blender

Starting to read 

In [3]:
args.use_AE = False

In [4]:
for batch in tqdm(train_loader):
    #t += 1
    ids, objs, boxes, triples, angles, attributes, obj_to_img, triple_to_img = tensor_aug(batch)
    z = torch.randn(objs.size(0), 64).to(objs.device)
    break

/home/yizhou/Research/3D_SLN/data/suncg_dataset.py:185: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  real_objs = (objs != __room__).nonzero().squeeze(1)


/home/yizhou/Research/3D_SLN/data/suncg_dataset.py:185: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  real_objs = (objs != __room__).nonzero().squeeze(1)


In [5]:
from torch_geometric.nn import RGCNConv
from torch_scatter import scatter_mean

In [6]:
kwargs = {
        'vocab': dt.vocab,
        'batch_size': args.batch_size,
        'train_3d': args.train_3d,
        'decoder_cat': args.decoder_cat,
        'embedding_dim': args.embedding_dim,
        'gconv_mode': args.gconv_mode,
        'gconv_num_layers': args.gconv_num_layers,
        'mlp_normalization': args.mlp_normalization,
        'vec_noise_dim': args.vec_noise_dim,
        'layout_noise_dim': args.layout_noise_dim,
        'use_AE': args.use_AE
    }

In [7]:
model = RGCNDecoder(**kwargs)

In [8]:
model = model.cuda()

In [9]:
z.shape

torch.Size([226, 64])

In [15]:
output = model.decoder(z, objs, triples, attributes)

0 obj_vecs torch.Size([226, 128])
edges torch.Size([2, 445])
1 obj_vecs torch.Size([226, 128])
edges torch.Size([2, 445])
2 obj_vecs torch.Size([226, 128])
edges torch.Size([2, 445])
3 obj_vecs torch.Size([226, 128])
edges torch.Size([2, 445])
4 obj_vecs torch.Size([226, 128])
edges torch.Size([2, 445])


In [18]:
output[0].shape

torch.Size([226, 6])

In [19]:
output[1].shape

torch.Size([226, 24])

In [11]:
s, p, o = triples.chunk(3, dim=1)  # All have shape (T, 1)
s, p, o = [x.squeeze(1) for x in [s, p, o]]  # Now have shape (T,)
edges = torch.stack([s, o], dim=1)  # Shape is (T, 2)

In [12]:
objs.shape

torch.Size([226])

In [20]:
edges.shape

torch.Size([445, 2])

In [32]:
boxes_pred = output[0]

In [23]:
dt.vocab

{'object_idx_to_name': ['__room__',
  'curtain',
  'shower_curtain',
  'dresser',
  'counter',
  'bookshelf',
  'picture',
  'mirror',
  'floor_mat',
  'chair',
  'sink',
  'desk',
  'table',
  'lamp',
  'door',
  'clothes',
  'person',
  'toilet',
  'cabinet',
  'floor',
  'window',
  'blinds',
  'wall',
  'pillow',
  'whiteboard',
  'bathtub',
  'television',
  'night_stand',
  'sofa',
  'refridgerator',
  'bed',
  'shelves'],
 'object_name_to_idx': {'__room__': 0,
  'curtain': 1,
  'shower_curtain': 2,
  'dresser': 3,
  'counter': 4,
  'bookshelf': 5,
  'picture': 6,
  'mirror': 7,
  'floor_mat': 8,
  'chair': 9,
  'sink': 10,
  'desk': 11,
  'table': 12,
  'lamp': 13,
  'door': 14,
  'clothes': 15,
  'person': 16,
  'toilet': 17,
  'cabinet': 18,
  'floor': 19,
  'window': 20,
  'blinds': 21,
  'wall': 22,
  'pillow': 23,
  'whiteboard': 24,
  'bathtub': 25,
  'television': 26,
  'night_stand': 27,
  'sofa': 28,
  'refridgerator': 29,
  'bed': 30,
  'shelves': 31},
 'pred_idx_to_na

In [24]:
s, p, o = triples.chunk(3, dim=1)  # All have shape (T, 1)
s, p, o = [x.squeeze(1) for x in [s, p, o]]  # Now have shape (T,)


In [28]:
parent_relation_type_index = 15

In [33]:
visited_subject = []
for i in range(len(p)):
    if p[i] == parent_relation_type_index:
        subject_index = s[i].item()
        object_index = o[i].item()
        if subject_index not in visited_subject:
            boxes_pred[subject_index] += boxes_pred[object_index]
            visited_subject.append(subject_index)

In [38]:
boxes_pred.data

tensor([[-0.1439, -0.0662, -0.3018,  0.1239, -0.5238,  0.4261],
        [-0.2018,  0.5236, -0.2216,  0.4327, -0.1895,  0.1938],
        [ 0.0211,  0.2248,  0.0313, -0.1986,  0.4344,  0.1646],
        ...,
        [ 0.0263,  0.0718, -0.1456, -0.3007, -0.2938, -0.3638],
        [-0.5394, -0.6079, -0.0038, -0.5237,  0.5197,  0.2469],
        [-0.2803,  0.2048,  0.0850, -0.1053, -0.1157, -0.1020]],
       device='cuda:0')